# Trabalho 2 - Tópicos em IA
#### Aluno: Matheus Bernard Mota
#### RGA: 2022.1904.008-6

---



## Eficácia de Diferentes Técnicas de Pooling na Detecção de Fake News

Neste notebook, utilizaremos o modelo BERT pré-treinado em pt-br **BERTimbau** com uma camada linear para classificação.
Para o Fine-tuning, utilizaremos o dataset de *fake-news* Fake.br corpus. A ideia é, a partir do modelo pré treinado, especializá-lo em: a partir de uma sequência (notícia), corretamente predizer se ela é verdadeira **(true)** ou falsa, **(fake)**.

O código fonte foi modularizado para melhor organização, compreensão e manutenibilidade e um notebook explicativo foi posto para direcionar a execução do procedimento e análise. 

---

## Procedimentos Iniciais
1. Anexar a pasta .src com este notebook, para que ele tenha acesso aos seus módulos
2. Realizar os imports iniciais
3. Mostra o device aceito pelo torch (#visualizações_adicionais)

In [2]:
#permite importar src

import os, sys
# import sys

root_relative_path = '../'
root = os.path.abspath(root_relative_path)

sys.path.append(root)

In [ ]:
#importa tudo aquilo que importa
import torch
import random
import numpy as np
import evaluate

from src.dataset_loader import DatasetLoader
from src.custom_bertimbau_classifier import CustomBertimbauClassifier
from src.baseline_bertimbau_classifier import BaselineBertimbauClassifier
from src.fine_tuner import FineTuner
import src.config as cfg


c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#physics are relative, but this code gonna be DETERMINISTIC BABY!
random.seed(cfg.RANDOM_SEED) 
np.random.seed(cfg.RANDOM_SEED) 
torch.manual_seed(cfg.RANDOM_SEED)

torch.backends.cudnn.deterministic = True

In [ ]:
print(cfg.DEVICE)

cuda


## Carregando o Dataset de Notícias

O dataset contém 7200 notícias, com labels num ratio de ~50%

#### Under the hood

O que este loader está fazendo?
1. Acessando o arquivo **pre-processed.csv** de um clone do repositório do Fake.br. (está no .gitignore, em caso de configuração local, este detalhe deve ser levado em conta)
2. Carregando um tokenizer, proveniente do próprio bertimbau, para traduzir o texto do corpus em tokens processados 
3. Separando as colunas dos textos e suas respectivas "labels", ou seja, classificação da veracidade da notícia
4. Separando, destas colunas, uma porcentagem para treino, validação(ajuste manual de parâmetros) e teste
5. transformando estes dados em tokens e preparando um dataset anexavel ao Trainer da biblioteca Transformers

In [ ]:
path = os.path.join(root, cfg.PATH_TO_DATASET)
ds_loader = DatasetLoader(
    path=path,
    model_name=cfg.BERTIMBAU,
    max_len=cfg.SEQ_LEN
)

#conjunto para teste, validação e treino
train_dataset, val_dataset, test_dataset = (ds_loader
                                            .load_dataset(seed=cfg.RANDOM_SEED)
                                            .get_datasets())



In [ ]:
#apenas para checar se tá tudo ok
print(train_dataset.encodings)
print(train_dataset[0])
print(repr(train_dataset))

## Carregando modelos BERTimbau

#### **baseline_model**
Versão do modelo onde o pooling é feito pela passagem do token **[CLS]**, presente no início de cada sentença, de modo a capturar o contexto geral de todo o documento. Este vetor é utilizado como entrada na camada de classificação.

#### **alternative_model**
Similarmente ao **baseline_model**, este modelo também aproveita do contexto capturado pelo token **[CLS]**, mas ele é concatenado com um vetor que calcula a média dos valores de todos os tokens presentes em cada sentença. Ou seja, se tenho X senteças de tamanho n  (os tamanhos variam, mas isso é regularizado com o token [PAD], que está configurado para não ser levado em consideração nas manipulações matriciais), cada i-ésimo token dos n tokens de cada sentença é somado X vezes, e uma média é retirada, divindindo o valor pelos numero de tokens significativos somados. Essa é outra forma de realizar o pooling, e garante um vetor de tamanho igual a de [CLS]. Estes dois vetores são então concatenados (dobrando o tamanho das features) e levado para a camada de classificação.

$$\text{mean\_pooling}(\mathbf{H}, \mathbf{M}) = \frac{\sum_{i=1}^{n} \mathbf{h}_i \cdot m_i}{\sum_{i=1}^{n} m_i}$$

onde:
- $\mathbf{H}$ é a matriz de embedding
- $\mathbf{M}$ é a matriz de atenção, responsável por eliminar os tokens [PAD] da conta 

$$\text{concat\_pooling}(\mathbf{H}, \mathbf{M}) = [\mathbf{h}_{[CLS]}; \text{mean\_pooling}(\mathbf{H}, \mathbf{M})]$$



In [ ]:
model_kwargs = ds_loader.get_labels_mapping()

alternative_model = CustomBertimbauClassifier(cfg.BERTIMBAU, **model_kwargs).to(cfg.DEVICE)
baseline_model = BaselineBertimbauClassifier(cfg.BERTIMBAU, **model_kwargs).to(cfg.DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

print("Configuraç~ao dos Modelos:")
print(baseline_model)
print("___________________________________________")
print(alternative_model)

## Fine tunando os modelos

In [ ]:
#retirado de original_script
try:
    acc_metric = evaluate.load("accuracy")
    f1_metric  = evaluate.load("f1")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        r1 = acc_metric.compute(predictions=preds, references=labels) #suspeito se ele vai acessar acc_metric ou nao
        r2 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
        return {"accuracy": r1["accuracy"], "f1": r2["f1"]}
except Exception:
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = (preds == labels).mean()
        return {"accuracy": float(acc)}


In [ ]:

fine_tuner = FineTuner(
    tokenizer=ds_loader.get_tokenizer())

tuned_baseline_res = (fine_tuner
                        .set_compute_metrics(compute_metrics)
                        .set_training_arguments(
                            output_dir=os.path.join(root, "bertimbau-baseline-cls-ptbr"),
                            evaluation_strategy="epoch",
                            save_strategy="no",
                            learning_rate=cfg.LEARNING_RATE,
                            weight_decay=0.01,
                            per_device_train_batch_size=cfg.BATCH_SIZE,
                            per_device_eval_batch_size=cfg.BATCH_SIZE,
                            num_train_epochs=cfg.NUM_EPOCHS,
                            fp16=torch.cuda.is_available(),
                            logging_steps=50,
                            report_to="none",
                            seed=cfg.RANDOM_SEED
                        )
                        .set_trainer(
                            model=baseline_model,
                            train_dataset=train_dataset,
                            eval_dataset=val_dataset
                        )
                        .train())

NameError: name 'FineTuner' is not defined

In [ ]:

#bem que eu poderia criar um método de reset
fine_tuner = FineTuner(
    tokenizer=ds_loader.get_tokenizer())

tuned_baseline_res = (fine_tuner
                        .set_compute_metrics(compute_metrics)
                        .set_training_arguments(
                            output_dir=os.path.join(root, "bertimbau-alternativo-cls-ptbr"),
                            evaluation_strategy="epoch",
                            save_strategy="no",
                            learning_rate=cfg.LEARNING_RATE,
                            weight_decay=0.01,
                            per_device_train_batch_size=cfg.BATCH_SIZE,
                            per_device_eval_batch_size=cfg.BATCH_SIZE,
                            num_train_epochs=cfg.NUM_EPOCHS,
                            fp16=torch.cuda.is_available(),
                            logging_steps=50,
                            report_to="none",
                            seed=cfg.RANDOM_SEED
                        )
                        .set_trainer(
                            model=alternative_model,
                            train_dataset=train_dataset,
                            eval_dataset=val_dataset
                        )
                        .train())

[Aviso] parâmetros ignorados nesta versão: ['evaluation_strategy']


c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\src\fine_tuner.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self._trainer = Trainer(


AcceleratorError: CUDA error: no kernel image is available for execution on the device
Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
